In [1]:
# 1、load
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

In [2]:
# 2、Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [3]:
# 3、Store
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

Found model file at  /home/zk/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin


In [4]:
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import GPT4All

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)

llm = GPT4All(model="./models/ggml-gpt4all-j-v1.3-groovy.bin")
# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "Summarize this content: {context}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context"
)

Found model file at  ./models/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from './models/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [5]:
from langchain.chains import (
    LLMChain, ConversationalRetrievalChain
)
from langchain.prompts import PromptTemplate

template = (
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {chat_history}"
    "Follow up question: {question}"
)
prompt = PromptTemplate.from_template(template)
question_generator_chain = LLMChain(llm=llm, prompt=prompt)

chain = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    question_generator=question_generator_chain,
    combine_docs_chain=combine_docs_chain,
)

In [8]:
chat_history = []
query = "What are some of the main ideas in self-reflection?"
result = chain({"question": query, "chat_history": chat_history})

In [9]:
result

{'question': 'What are some of the main ideas in self-reflection?',
 'chat_history': [],
 'answer': " The goal of LLM in real world tasks like robotics, game AI and autonomous driving systems is to learn how to make decisions based on its own experiences rather than relying solely on preprogrammed rules or heuristics. This allows for more adaptability and flexibility in decision making as well as the ability to handle unforeseen situations that may arise during a task execution.\nSelf-reflection refers to an agent's process of examining past actions, their outcomes, and any mistakes made along the way. It is essential for autonomous agents to improve iteratively by refining previous action decisions and correcting errors in order to achieve better results over time. Self-reflection plays a crucial role in real-world tasks where trial and error are inevitable.\nSelf-reflection can be achieved through showing two-shot examples of failed trajectories paired with ideal reflections for guid

In [11]:
chain({"question": "How does the Reflexion paper handle it?", "chat_history": chat_history})

{'question': 'How does the Reflexion paper handle it?',
 'chat_history': [],
 'answer': '\nThe agent can reset the environment by calling `reset()` function, which resets all variables and returns them to their initial values. The agent then continues with its next action based on LLM��s output.'}